# Ingestion de fichiers en tables Iceberg
Ce notebook parcourt récursivement un dossier, lit chaque fichier (CSV ou Parquet) et crée une table Iceberg dans un catalogue S3 (MinIO).

In [ ]:
from pyspark.sql import SparkSession

# -----------------------------------------------------
# 1. Démarrage de la session Spark
# -----------------------------------------------------
spark = SparkSession.builder.appName("IcebergNotebook").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
print("Spark session initialisée avec Iceberg et MinIO.")

In [ ]:
# -----------------------------------------------------
# 2. Initialisation boto3 client MinIO
# -----------------------------------------------------

import boto3

s3 = boto3.client(
    "s3",
    endpoint_url="http://minio:9000",
    aws_access_key_id="minio",
    aws_secret_access_key="minio123",
    region_name="eu-west-1",
)

bucket = "raw"
prefix = "unzipped/"

print(f"Listing des fichiers dans s3://{bucket}/{prefix}")

response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

if "Contents" not in response:
    print("Aucun fichier trouvé.")
    raise SystemExit

files = [obj["Key"] for obj in response["Contents"] if not obj["Key"].endswith("/")]

print("Fichiers trouvés :", files)


In [ ]:
# -----------------------------------------------------
# 3. Ingestion dans Iceberg
# -----------------------------------------------------
spark.sql('CREATE NAMESPACE IF NOT EXISTS lakehouse.raw').show()
print(f"Namespace Iceberg créée : raw")

for key in files:
    filename = key.split("/")[-1]

    if not (filename.endswith(".csv") or filename.endswith(".parquet")):
        print(f"Format non supporté : {filename}")
        continue

    table_name = filename.split(".")[0].replace("-", "_").replace(" ", "_")

    file_path = f"s3a://{bucket}/{key}"

    # Lecture Spark
    if filename.endswith(".csv"):
        df = spark.read.option("header", "true").csv(file_path)
    else:
        df = spark.read.parquet(file_path)

    full_table_name = f"lakehouse.raw.{table_name}"
    df.writeTo(full_table_name).createOrReplace()

    print(f"Table Iceberg créée : {full_table_name}")
print("🎯 Toutes les opérations ont été exécutées.")